In [1]:
import pandas as pd
import os
import json
import geojson

In [2]:
with open('Resources/RAW/transit_stops_datasd.geojson') as f:
    json_data = json.load(f)

#print(json_data)

In [3]:
Df = pd.DataFrame(json_data['features'])
Df.head()

,type,properties,geometry
0,Feature,"{'objectid': 1, 'stop_uid': 'MTS_10326', 'stop...","{'type': 'Point', 'coordinates': [-117.0195758..."
1,Feature,"{'objectid': 2, 'stop_uid': 'MTS_10328', 'stop...","{'type': 'Point', 'coordinates': [-117.0154153..."
2,Feature,"{'objectid': 3, 'stop_uid': 'MTS_10330', 'stop...","{'type': 'Point', 'coordinates': [-117.0127881..."
3,Feature,"{'objectid': 4, 'stop_uid': 'MTS_10331', 'stop...","{'type': 'Point', 'coordinates': [-117.0109914..."
4,Feature,"{'objectid': 5, 'stop_uid': 'MTS_10332', 'stop...","{'type': 'Point', 'coordinates': [-117.0092430..."


In [4]:
json_data['features']

[{'type': 'Feature',
  'properties': {'objectid': 1,
   'stop_uid': 'MTS_10326',
   'stop_id': '10326',
   'stop_code': 10326,
   'stop_name': 'Allison Av & Palm Av',
   'stop_lat': 32.765915,
   'stop_lon': -117.019588,
   'stop_agncy': 'MTS',
   'wheelchair': 1,
   'intersec': 'N-E/B',
   'stop_place': 'allpal',
   'parent_sta': None,
   'lat': 32.76591052622188,
   'lng': -117.01957585450016},
  'geometry': {'type': 'Point',
   'coordinates': [-117.01957585450013, 32.76591052536231]}},
 {'type': 'Feature',
  'properties': {'objectid': 2,
   'stop_uid': 'MTS_10328',
   'stop_id': '10328',
   'stop_code': 10328,
   'stop_name': 'La Mesa Bl & University Av',
   'stop_lat': 32.767805,
   'stop_lon': -117.015427,
   'stop_agncy': 'MTS',
   'wheelchair': 1,
   'intersec': 'N-N/B',
   'stop_place': None,
   'parent_sta': None,
   'lat': 32.76779987540056,
   'lng': -117.01541535900714},
  'geometry': {'type': 'Point',
   'coordinates': [-117.01541535900711, 32.767799874540955]}},
 {'type':

In [5]:
prop_df = pd.DataFrame([x['properties'] for x in json_data['features']])
prop_df.head()

,objectid,stop_uid,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_agncy,wheelchair,intersec,stop_place,parent_sta,lat,lng
0,1,MTS_10326,10326,10326,Allison Av & Palm Av,32.765915,-117.019588,MTS,1,N-E/B,allpal,None,32.765911,-117.019576
1,2,MTS_10328,10328,10328,La Mesa Bl & University Av,32.767805,-117.015427,MTS,1,N-N/B,None,None,32.767800,-117.015415
2,3,MTS_10330,10330,10330,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800,MTS,1,F-S/B,None,None,32.714223,-117.012788
3,4,MTS_10331,10331,10331,La Mesa Bl & Glen St,32.769344,-117.011003,MTS,1,N-E/B,None,None,32.769339,-117.010991
4,5,MTS_10332,10332,10332,Orville St & Brucker Av,32.709168,-117.009255,MTS,1,M-Street-E/B,orvbru,None,32.709163,-117.009243


In [6]:
properties_df = pd.DataFrame(prop_df, index=None, columns=['stop_name', 'stop_lat', 'stop_lon'])
properties_df.head()

,stop_name,stop_lat,stop_lon
0,Allison Av & Palm Av,32.765915,-117.019588
1,La Mesa Bl & University Av,32.767805,-117.015427
2,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800
3,La Mesa Bl & Glen St,32.769344,-117.011003
4,Orville St & Brucker Av,32.709168,-117.009255


In [8]:
lats = properties_df['stop_lat']
lons = properties_df['stop_lon']
plots = zip(lats,lons)
cords = list(plots)
#print(cords)

In [12]:
properties_df.rename(columns={'stop_lat':'lat', 'stop_lon':'lng'}, inplace=True)
properties_df

,stop_name,lat,lng
0,Allison Av & Palm Av,32.765915,-117.019588
1,La Mesa Bl & University Av,32.767805,-117.015427
2,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800
3,La Mesa Bl & Glen St,32.769344,-117.011003
4,Orville St & Brucker Av,32.709168,-117.009255
...,...,...,...
6214,Plaza Camino Real Transit Center,33.178276,-117.336525
6215,San Luis Rey Transit Center,33.254410,-117.298129
6216,Solana Beach Station,32.992937,-117.271067
6217,Sorrento Valley Station,32.902813,-117.225088


In [9]:
import geopy
from geopy.geocoders import Nominatim

In [13]:

# getting geoData

def get_zipcode(df, geolocator, lat_field, lon_field):
    location = geolocator.reverse((df[lat_field], df[lon_field]))
    return location.raw['address']


geolocator = geopy.Nominatim(user_agent='user-agent')

zipcodes = properties_df.apply(get_zipcode, axis=1, geolocator=geolocator, lat_field='lat', lon_field='lng')

In [14]:
# Export dataframe as js file 
output_file = 'transit.js'
zipcodes.to_json(output_file, orient='records')

In [15]:
# Reading the js file and exporting to dataframe
zips = pd.read_json('./transit.js')
zips = pd.DataFrame(zips)
zips

,house_number,road,town,county,state,ISO3166-2-lvl4,postcode,country,country_code,amenity,...,craft,railway,city_district,place,residential,aeroway,commercial,man_made,industrial,military
0,4757,Palm Avenue,La Mesa,San Diego County,California,US-CA,91941.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Crowder Lane,La Mesa,San Diego County,California,US-CA,91941.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1047,Sweetwater Road,NaN,San Diego County,California,US-CA,91977.0,United States,us,Jack in the Box,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,8665,La Mesa Boulevard,La Mesa,San Diego County,California,US-CA,91941.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8802,Orville Street,NaN,San Diego County,California,US-CA,91977.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6214,NaN,South Vista Way,NaN,San Diego County,California,US-CA,92008.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6215,NaN,North River Road,NaN,San Diego County,California,US-CA,92057.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6216,NaN,North Cedros Avenue,Solana Beach,San Diego County,California,US-CA,92075.0,United States,us,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6217,NaN,Sorrento Valley Road,NaN,San Diego County,California,US-CA,92092.0,United States,us,NaN,...,NaN,Sorrento Valley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Checking the columns of the new dataframe
zips.columns

Index(['house_number', 'road', 'town', 'county', 'state', 'ISO3166-2-lvl4',
       'postcode', 'country', 'country_code', 'amenity', 'suburb', 'city',
       'highway', 'neighbourhood', 'shop', 'tourism', 'state_district',
       'village', 'building', 'retail', 'office', 'leisure', 'healthcare',
       'quarter', 'historic', 'district', 'hamlet', 'craft', 'railway',
       'city_district', 'place', 'residential', 'aeroway', 'commercial',
       'man_made', 'industrial', 'military'],
      dtype='object')

In [17]:
# Keeping relevant columns
transit_zipcodes = zips[['postcode', 'county']]
transit_zipcodes = pd.DataFrame(transit_zipcodes)

In [41]:
# concating the two dataframes
final_transit_df = pd.concat([properties_df, transit_zipcodes], axis = 1)
final_transit_df

,stop_name,lat,lng,postcode,county
0,Allison Av & Palm Av,32.765915,-117.019588,91941.0,San Diego County
1,La Mesa Bl & University Av,32.767805,-117.015427,91941.0,San Diego County
2,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800,91977.0,San Diego County
3,La Mesa Bl & Glen St,32.769344,-117.011003,91941.0,San Diego County
4,Orville St & Brucker Av,32.709168,-117.009255,91977.0,San Diego County
...,...,...,...,...,...
6214,Plaza Camino Real Transit Center,33.178276,-117.336525,92008.0,San Diego County
6215,San Luis Rey Transit Center,33.254410,-117.298129,92057.0,San Diego County
6216,Solana Beach Station,32.992937,-117.271067,92075.0,San Diego County
6217,Sorrento Valley Station,32.902813,-117.225088,92092.0,San Diego County


In [42]:
# Removing NaN or other non-numeric values from the postcode column
transit_df = final_transit_df[pd.to_numeric(final_transit_df['postcode'], errors='coerce').notnull()]
transit_df

,stop_name,lat,lng,postcode,county
0,Allison Av & Palm Av,32.765915,-117.019588,91941.0,San Diego County
1,La Mesa Bl & University Av,32.767805,-117.015427,91941.0,San Diego County
2,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800,91977.0,San Diego County
3,La Mesa Bl & Glen St,32.769344,-117.011003,91941.0,San Diego County
4,Orville St & Brucker Av,32.709168,-117.009255,91977.0,San Diego County
...,...,...,...,...,...
6214,Plaza Camino Real Transit Center,33.178276,-117.336525,92008.0,San Diego County
6215,San Luis Rey Transit Center,33.254410,-117.298129,92057.0,San Diego County
6216,Solana Beach Station,32.992937,-117.271067,92075.0,San Diego County
6217,Sorrento Valley Station,32.902813,-117.225088,92092.0,San Diego County


In [43]:
# getting rid of the decimal
transit_df['postcode'] = transit_df['postcode'].astype(int)
transit_df

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_90338/4087725392.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transit_df['postcode'] = transit_df['postcode'].astype(int)


,stop_name,lat,lng,postcode,county
0,Allison Av & Palm Av,32.765915,-117.019588,91941,San Diego County
1,La Mesa Bl & University Av,32.767805,-117.015427,91941,San Diego County
2,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800,91977,San Diego County
3,La Mesa Bl & Glen St,32.769344,-117.011003,91941,San Diego County
4,Orville St & Brucker Av,32.709168,-117.009255,91977,San Diego County
...,...,...,...,...,...
6214,Plaza Camino Real Transit Center,33.178276,-117.336525,92008,San Diego County
6215,San Luis Rey Transit Center,33.254410,-117.298129,92057,San Diego County
6216,Solana Beach Station,32.992937,-117.271067,92075,San Diego County
6217,Sorrento Valley Station,32.902813,-117.225088,92092,San Diego County


In [44]:
transit_df.rename(columns={'postcode':'Zip Code'}, inplace=True)
transit_df

/var/folders/16/tnbbfsrj49l5pz272sb2fgt40000gn/T/ipykernel_90338/1153012688.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transit_df.rename(columns={'postcode':'Zip Code'}, inplace=True)


,stop_name,lat,lng,Zip Code,county
0,Allison Av & Palm Av,32.765915,-117.019588,91941,San Diego County
1,La Mesa Bl & University Av,32.767805,-117.015427,91941,San Diego County
2,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800,91977,San Diego County
3,La Mesa Bl & Glen St,32.769344,-117.011003,91941,San Diego County
4,Orville St & Brucker Av,32.709168,-117.009255,91977,San Diego County
...,...,...,...,...,...
6214,Plaza Camino Real Transit Center,33.178276,-117.336525,92008,San Diego County
6215,San Luis Rey Transit Center,33.254410,-117.298129,92057,San Diego County
6216,Solana Beach Station,32.992937,-117.271067,92075,San Diego County
6217,Sorrento Valley Station,32.902813,-117.225088,92092,San Diego County


In [45]:
# Set the 'Zip Code' column as the index
transit_df.set_index('Zip Code', inplace=True)
transit_df

,stop_name,lat,lng,county
Zip Code,,,,
91941,Allison Av & Palm Av,32.765915,-117.019588,San Diego County
91941,La Mesa Bl & University Av,32.767805,-117.015427,San Diego County
91977,Sweetwater Rd & Jamacha Rd,32.714228,-117.012800,San Diego County
91941,La Mesa Bl & Glen St,32.769344,-117.011003,San Diego County
91977,Orville St & Brucker Av,32.709168,-117.009255,San Diego County
...,...,...,...,...
92008,Plaza Camino Real Transit Center,33.178276,-117.336525,San Diego County
92057,San Luis Rey Transit Center,33.254410,-117.298129,San Diego County
92075,Solana Beach Station,32.992937,-117.271067,San Diego County


In [46]:
# exporting file
from pathlib import Path

filepath = Path("Resources/Clean/San_Diego_Transit.csv")
transit_df.to_csv(filepath)